##### Copyright 2018 The TensorFlowAuthors。

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# トレーニングチェックポイント

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.orgで表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">GoogleColabで実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

「TensorFlowモデルを保存する」というフレーズは、通常、次の2つのいずれかを意味します。

1. チェックポイント、または
2. SavedModel。

チェックポイントは、モデルで使用されるすべてのパラメーター（ `tf.Variable`オブジェクト）の正確な値をキャプチャします。チェックポイントには、モデルによって定義された計算の説明が含まれていないため、通常、保存されたパラメーター値を使用するソースコードが利用可能な場合にのみ役立ちます。

一方、SavedModel形式には、パラメーター値（チェックポイント）に加えて、モデルによって定義された計算のシリアル化された説明が含まれます。この形式のモデルは、モデルを作成したソースコードから独立しています。したがって、TensorFlow Serving、TensorFlow Lite、TensorFlow.js、または他のプログラミング言語（C、C ++、Java、Go、Rust、C＃などのTensorFlow API）のプログラムを介したデプロイに適しています。

このガイドでは、チェックポイントの書き込みと読み取りのためのAPIについて説明します。

## セットアップ

In [0]:
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [0]:
net = Net()

## `tf.keras`トレーニングAPIからの保存

See the [`tf.keras` guide on saving and restoring](./keras/overview.ipynb#save_and_restore).

`tf.keras.Model.save_weights`は、TensorFlowチェックポイントを保存します。 

In [0]:
net.save_weights('easy_checkpoint')

## チェックポイントの作成


TensorFlowモデルの永続的な状態は、 `tf.Variable`オブジェクトに保存されます。これらは直接構築できますが、多くの場合、 `tf.keras.layers`や`tf.keras.Model`などの高レベルAPIを介して作成されます。

変数を管理する最も簡単な方法は、変数をPythonオブジェクトにアタッチしてから、それらのオブジェクトを参照することです。

`tf.train.Checkpoint` 、 `tf.keras.layers.Layer` 、および`tf.keras.Model`サブクラスは、それらの属性に割り当てられた変数を自動的に追跡します。次の例では、単純な線形モデルを作成し、モデルのすべての変数の値を含むチェックポイントを書き込みます。

`Model.save_weights`てモデルチェックポイントを簡単に保存できます

### 手動チェックポイント

#### セットアップ

To help demonstrate all the features of `tf.train.Checkpoint` define a toy dataset and optimization step:

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [0]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### チェックポイントオブジェクトを作成します

チェックポイントを手動で作成するには、 `tf.train.Checkpoint`オブジェクトが必要です。チェックポイントするオブジェクトがオブジェクトの属性として設定されている場所。

`tf.train.CheckpointManager`は、複数のチェックポイントの管理にも役立ちます。

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### モデルのトレーニングとチェックポイント

次のトレーニングループは、モデルとオプティマイザーのインスタンスを作成し、それらを`tf.train.Checkpoint`オブジェクトに収集します。データの各バッチでループ内のトレーニングステップを呼び出し、定期的にチェックポイントをディスクに書き込みます。

In [0]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [0]:
train_and_checkpoint(net, manager)

#### トレーニングを復元して続行する

最初の後に、新しいモデルとマネージャーを渡すことができますが、中断したところから正確にトレーニングをピックアップします。

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

`tf.train.CheckpointManager`オブジェクトは、古いチェックポイントを削除します。上記では、最新の3つのチェックポイントのみを保持するように構成されています。

In [0]:
print(manager.checkpoints)  # List the three remaining checkpoints

これらのパス（例： `'./tf_ckpts/ckpt-10'` ）は、ディスク上のファイルではありません。代わりに、 `index`ファイルと変数値を含む1つ以上のデータファイルのプレフィックスです。これらのプレフィックスは、 `CheckpointManager`がその状態を保存する単一の`checkpoint`ファイル（ `'./tf_ckpts/checkpoint'` ）にグループ化されます。

In [0]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## 力学の読み込み

TensorFlow matches variables to checkpointed values by traversing a directed graph with named edges, starting from the object being loaded. Edge names typically come from attribute names in objects, for example the `"l1"` in `self.l1 = tf.keras.layers.Dense(5)`. `tf.train.Checkpoint` uses its keyword argument names, as in the `"step"` in `tf.train.Checkpoint(step=...)`.

上記の例の依存関係グラフは次のようになります。

![トレーニングループの例の依存関係グラフの視覚化](https://tensorflow.org/images/guide/whole_checkpoint.svg)

オプティマイザーは赤、通常の変数は青、オプティマイザーのスロット変数はオレンジです。たとえば`tf.train.Checkpoint`表す他のノードは、黒です。

スロット変数はオプティマイザーの状態の一部ですが、特定の変数用に作成されます。たとえば、上記の`'m'`エッジは、Adamオプティマイザが各変数に対して追跡する運動量に対応します。スロット変数は、変数とオプティマイザーの両方が保存される場合にのみチェックポイントに保存されます。つまり、破線のエッジです。

Calling `restore()` on a `tf.train.Checkpoint` object queues the requested restorations, restoring variable values as soon as there's a matching path from the `Checkpoint` object. For example we can load just the bias from the model we defined above by reconstructing one path to it through the network and the layer.

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # We get the restored value now

The dependency graph for these new objects is a much smaller subgraph of the larger checkpoint we wrote above. It includes only the bias and a save counter that `tf.train.Checkpoint` uses to number checkpoints.

![バイアス変数のサブグラフの視覚化](https://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore()`は、オプションのアサーションを持つステータスオブジェクトを返します。新しい`Checkpoint`作成したすべてのオブジェクトが復元されたため、 `status.assert_existing_objects_matched()`が渡されます。

In [0]:
status.assert_existing_objects_matched()

There are many objects in the checkpoint which haven't matched, including the layer's kernel and the optimizer's variables. `status.assert_consumed()` only passes if the checkpoint and the program match exactly, and would throw an exception here.

### 復元の遅延

`Layer` objects in TensorFlow may delay the creation of variables to their first call, when input shapes are available. For example the shape of a `Dense` layer's kernel depends on both the layer's input and output shapes, and so the output shape required as a constructor argument is not enough information to create the variable on its own. Since calling a `Layer` also reads the variable's value, a restore must happen between the variable's creation and its first use.

To support this idiom, `tf.train.Checkpoint` queues restores which don't yet have a matching variable.

In [0]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restored

### チェックポイントを手動で検査する

`tf.train.list_variables`は、チェックポイントキーとチェックポイント内の変数の形状を一覧表示します。チェックポイントキーは、上に表示されたグラフのパスです。

In [0]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

### リストと辞書の追跡

`self.l1 = tf.keras.layers.Dense(5)`ような直接の属性割り当てと同様に、リストと辞書を属性に割り当てると、その内容が追跡されます。

In [0]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

リストや辞書のラッパーオブジェクトに気付くかもしれません。これらのラッパーは、基になるデータ構造のチェックポイント可能なバージョンです。属性ベースのロードと同様に、これらのラッパーは、変数がコンテナーに追加されるとすぐに変数の値を復元します。

In [0]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

同じ追跡が`tf.keras.Model`サブクラスに自動的に適用され、たとえばレイヤーのリストを追跡するために使用できます。

## Estimatorを使用したオブジェクトベースのチェックポイントの保存

See the [Estimator guide](https://www.tensorflow.org/guide/estimator).

デフォルトでは、Estimatorは、前のセクションで説明したオブジェクトグラフではなく、変数名でチェックポイントを保存します。 `tf.train.Checkpoint`は名前ベースのチェックポイントを受け入れますが、モデルの一部をEstimatorの`model_fn`外に移動すると、変数名が変更される場合があります。オブジェクトベースのチェックポイントを保存すると、Estimator内でモデルをトレーニングし、モデルの外で使用することが簡単になります。

In [0]:
import tensorflow.compat.v1 as tf_compat

In [0]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Tell the Estimator to save "ckpt" in an object-based format.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

`tf.train.Checkpoint`は、 `model_dir`からEstimatorのチェックポイントをロードできます。

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # From est.train(..., steps=10)

## 概要

TensorFlowオブジェクトは、使用する変数の値を保存および復元するための簡単な自動メカニズムを提供します。
